Here is done in this file the biggest part of the project which is analyzing the data and mining it.

In [89]:
from lyrics_extractor import extract_all_artist

df = extract_all_artist()
df

,artist,title,lyrics
0,1.Cuz,Akta mannen,"Akta mannen Lyrics[Refräng]\nHar DJ med mig, a..."
1,1.Cuz,AKTA MANNEN (Remix),AKTA MANNEN (Remix) Lyrics[Intro]\nWahzaa\nYei...
2,1.Cuz,RÄKNA MINA DAGAR,RÄKNA MINA DAGAR Lyrics[Intro: 1.Cuz]\nKänns s...
3,1.Cuz,FIENDES FIENDE,FIENDES FIENDE Lyrics[Intro: 1.Cuz]\nDet kan v...
4,1.Cuz,Swedens Most Wanted,Swedens Most Wanted Lyrics[Intro]\nHehe (Grrr)...
...,...,...,...
5,Zacke,Utomlands,Utomlands Lyrics[Intro: Peter Morén]\nDet är s...
6,Zacke,Kommer hem,Kommer hem Lyrics[Vers 1: Zacke]\nFuckar upp a...
7,Zacke,Göra upp en eld,"Göra upp en eld Lyrics[Vers 1, Zacke]\nÄlskar ..."
8,Zacke,En glasspinne,En glasspinne Lyrics[Intro]\n(Det är allt\nDet...


In [90]:
df.value_counts('artist')

artist
1.Cuz                10
Promoe               10
Looptroop Rockers    10
Mange Schmidt        10
Maskinen             10
Medina               10
Mohammed Ali         10
Movits!              10
Newkid               10
Näääk (SWE)          10
Petter               10
Roffe Ruff           10
ADAAM                10
STOR                 10
Silvana Imam         10
Snook                10
Svenska Akademien    10
The Latin Kings      10
Thomas Rusiak        10
Timbuktu             10
VC Barre             10
Yasin                10
Linda Pira           10
Lilla Namo           10
Lazee                10
Labyrint             10
Adam Tensta          10
Afasi & Filthy       10
Ant Wan              10
Cleo (SWE)           10
Dani M               10
Dree Low             10
Ecco2k               10
Einár                10
Fronda               10
Greekazo             10
Hov1                 10
Infinite Mass        10
Just D               10
Kartellen            10
Ken Ring             10
Zacke    

We see that for some artists (StoneFunkers, Kumba, Jiggz, Highwon and Addis Black Widow) we couldn't collect 10 songs beause they provided less number of songs on www.genius.com .

I could have drop these artists from the dataframe to get a really smooth data distribution, however, i'm interested to see how this data points are going to be computed by the program, so i decided to let these artists in the dataset.

In [91]:
import spacy
nlp_sv = spacy.load("sv_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

#IL FAUT PASSER LE TEXTE EN MINUSCULE MAIS FAIRE ATTENTION AUX EXCEPTIONS TYPE : INT.lower() doesn't work.

#Add contextual stopwords to our swedish stopwords list.
for stopword in ["refräng", "vers", "lyrics", "intro", "outro"]:
    nlp_sv.Defaults.stop_words.add(stopword)

def preprocess_text(text):

    #REMOVING THE FIRST LINE
    text = text.split('\n', 1)[1]

    #TOKENISATION
    tokens =[]
    for token in nlp_sv(text):
        tokens.append(token)

    #REMOVING STOP WORDS
    spacy_stopwords_sv = nlp_sv.Defaults.stop_words
    spacy_stopwords_en = nlp_en.Defaults.stop_words
    sentence =  [word for word in tokens if word.text.isalpha() and word.text not in spacy_stopwords_sv and word.text not in spacy_stopwords_en]

    return sentence

In [92]:
df['lyrics'] = [preprocess_text(text) for text in df['lyrics']]
df['len(lyrics)'] = [len(lyrics) for lyrics in df['lyrics']]

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'isalpha'

In [ ]:
df

In [ ]:
import seaborn as sns
sns.histplot(data=df, x="title", y=len("lyrics"), binwidth=3)